## Hex GNN Experimentation

In [7]:
NUMAGENTS = 30
NUMEXPERIMENTS = 10
TIME = 25 # seconds
VISUAL = False

In [8]:
import simulation.Visualizer
import simulation.World
import simulation.Agent
import pickle
from matplotlib import pyplot as plt

simulation.Agent.USE = "SIM"
simulation.Agent.CASE = "HEX"

for i in range(NUMEXPERIMENTS):
    if i%10== 0:
        print(str(i)+"/"+str(NUMEXPERIMENTS))
    filename = "./data/hex-logs/experiment-"+str(i)+"-log.p"
    with open(filename, "wb") as f:
        world = simulation.World.World(f)
        for j in range(int(NUMAGENTS)):
            a = simulation.Agent.Agent(swarm=j%2)
            world.agents.append(a)

        if VISUAL:
            v = simulation.Visualizer.Visualizer(world)
            v.run()
        else:
            while world.stepCount < TIME*10:
                world.step()
            world.closeWorld()

0/10


In [9]:
import utils.HexDataset
import shutil

###############################
###  Gather and split data  ###
###############################

ta = 70 # Percentage used for training (remaining used for testing)

try:
    shutil.rmtree("/tmp/Hexs", ignore_errors=False, onerror=None)
except FileNotFoundError as e:
    pass
    
dataset = utils.HexDataset.HexDataset(root='/tmp/Hexs').shuffle()

train_data = dataset[0:int(ta/100.*len(dataset))]
test_data = dataset[int(ta/100.*len(dataset)):]

print(dataset[0])

Processing...
File: 0
File: 1
File: 2
File: 3
File: 4
File: 5
File: 6
File: 7
File: 8
File: 9
Done!
Data(edge_index=[2, 813], x=[30, 2], y=[30, 2])


In [10]:
from torch_geometric.data import Data, DataLoader
from utils.models import *
from tqdm import tqdm
import numpy as np
import torch
from time import sleep

np.random.seed(1337)

num_epochs = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainLoader = DataLoader(train_data, batch_size=32, shuffle=True)
testLoader = DataLoader(test_data, batch_size=32, shuffle=True)

node_dim = train_data[0].x.shape[1]
edge_dim = 0
if train_data[0].edge_attr: # if edge attributes used
    edge_dim = train_data[0].edge_attr.shape[1]
out_dim = train_data[0].y.shape[1]

model = GenericGNN(node_dim, edge_dim, out_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=4e-5, weight_decay=5e-7)

In [11]:
#from util import *
from random import random
from copy import deepcopy as copy
from tqdm import tqdm
from time import sleep
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

plt.rcParams['figure.figsize'] = (12, 4)

lossFunction = nn.MSELoss()

def train(model, optimizer, dataset):
    
    model.to(device)
    model.train()
    total_loss = 0.0
    num_items = 1
        
    for row in dataset:
        optimizer.zero_grad()
        pred = model(row.x, 
                     row.edge_index, 
                     row.edge_attr)
        loss = lossFunction(row.y, pred)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_items += 1
    return total_loss/num_items

def evaluate(model, dataset):
    
    model.to(device)
    model.eval()
    
    total_loss = 0.0
    num_items = 0
    
    num_outs = dataset[0].y.shape[1]
    
    predHist = [[] for _ in range(num_outs)]
    actualHist = [[] for _ in range(num_outs)]    

    for row in dataset:
        pred = model(row.x, 
                 row.edge_index, 
                 row.edge_attr)
        loss = lossFunction(row.y, pred)

        for i in range(len(pred[0])):
            predHist[i].append(pred[0][i].item())
            actualHist[i].append(row.y[0][i].item())
            
        total_loss += loss.item()
        num_items += 1
        
    return total_loss/num_items, actualHist, predHist

In [12]:
num_epochs = 200
epoch = 0
nbins  = 100
minmax = 0.5
bins  = [((i*(minmax*2))/nbins) - minmax for i in range(nbins+1)]
losses = []

plt.rcParams['figure.figsize'] = [5, 3]
plt.rcParams['figure.dpi'] = 100

for epoch in tqdm(range(epoch, num_epochs)):
       
    train_loss      = train(   model, optimizer, train_data)
    test_loss, a, p = evaluate(model,            test_data)
    
    losses.append([train_loss, test_loss])
    
    print("{train: " + str(train_loss) +"}, {test: " + str(test_loss) + "}")
    
    plt.rcParams['figure.figsize'] = (5, 3)    
    fig, ax = plt.subplots(1,1)
    fig.suptitle("Testing dataset", fontsize=16)
    ax.hist(p[0], bins=bins, alpha=0.5, label="predicted")
    ax.hist(a[0], bins=bins, alpha=0.5, label="actual")
    ax.legend()
    plt.show()
    
    torch.save(model.state_dict(), './models/hex-models/'+str(epoch)+'.mod')
    sleep(0.5)

  0%|          | 0/200 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.plot([i for i in range(len(losses))],[losses[i][0] for i in range(len(losses))])
plt.plot([i for i in range(len(losses))],[losses[i][1] for i in range(len(losses))])

In [ ]:
model.eval()

In [ ]:
import math

xs = 0.07
xe = 0.4

MULT = 1
TARGET_DIST = 0.15 
DIP = 1 

def lj(x, e):
    
    dist = math.sqrt(((x[1][0]-x[0][0])**2)+((x[1][1]-x[0][1])**2))
    epsilon = DIP*MULT
    sigma = (TARGET_DIST*MULT)/(2**(1/6))
    mag = min((epsilon*4) * (pow((sigma/dist),12)-pow((sigma/dist),6)), 10)
    
    angle = math.pi
    if mag > 0:
        angle = 0
    
    return mag, angle
    
    
numScatter = 5000
numPoints = 1000
delta = (1/numPoints)*(xe-xs)

plotDataX = []
plotDataPredY = []
plotDataPredA = []
plotDataPredX = []
plotDataActY = []
plotDataActA = []
plotDataEqnY = []


for i in range(numPoints):
    
    data = [[0, 0], [0, 0]]
    xVal = delta*i+delta + xs
    data[1] = [xVal, 0]
    plotDataX.append(xVal)
    
    e1 = [0, 1]
    e2 = [1, 0]
    
    x = torch.tensor(data, dtype=torch.float)
    e = torch.tensor([e1, e2], dtype=torch.long)
    
    actualMag, actualAng = lj(x, e)
    
    plotDataActY.append(actualMag)
    plotDataActA.append(math.degrees(actualAng))
    
pois = []

for i in range(numScatter):
    
    data = [[0, 0] for _ in range(20)]
    ox = random()
    oy = random()
    d = (random()*(xe-xs)+xs)
    a = random() * (2*math.pi)
    no = ox + d*math.cos(a)
    ny = oy + d*math.sin(a)
    
    data[0] = [ox, oy]
    data[1] = [no, ny]
    e1 = [0, 1]
    e2 = [1, 0]
    
    x = torch.tensor(data, dtype=torch.float)
    e = torch.tensor([e1, e2], dtype=torch.long)
    pred = model(x, e)
    ljMag, ljAng = lj(x, e)
    pois.append([ox, oy, no, ny, pred[0][0], pred[0][1], d])
    
    mag = math.sqrt((pred[0][0])**2 + (pred[0][1])**2)
    ang = abs(math.degrees(math.atan2(pred[0][1], pred[0][0]) - math.atan2(oy-ny, ox-no)))
    
    actAng  = math.degrees(ljAng)
       
    if ang < 90: # clip ang to 0
        mag = mag
    else: # clip ang to 180
        mag = -mag
    
    plotDataPredX.append(d)
    plotDataPredA.append(abs(actAng-ang))
    plotDataPredY.append(mag)
#    plotDataEqnY.append(eqnMag)

plt.rcParams['figure.figsize'] = [12, 3]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

fig, (ax1, ax2) = plt.subplots(1,2)

ys = -1.5* MULT
ye = 1.2*MULT

ax1.plot(plotDataX, plotDataActY, label='Actual Output from LJ Potential')
ax1.scatter(plotDataPredX, plotDataPredY, s=1, color=[1, 0, 0], alpha=0.1, label='Predicted by GNN of 20 agents')
ax1.set_xlim([xs, xe])
ax1.set_ylim([ys, ye])
ax1.plot([xs, xe], [0, 0], 'k--')
ax1.legend()
ax1.set_xlabel("Distance Between Two Agents (World Units)")
ax1.set_ylabel("Magnitude of Acceleration")

pickle.dump({'dist':plotDataPredX, 'mag':plotDataPredY}, open("GNNOutputData.p", 'wb'))

ax2.plot(plotDataX, plotDataActA, 'k--')
ax2.scatter(plotDataPredX, plotDataPredA, s=1, color=[1, 0, 0], alpha=0.1, label='Predicted by GNN of 20 agents')
ax2.set_xlim([xs, xe])
ax2.set_ylim([-5, 185])
ax2.plot([xs, xe], [0, 0], 'k--')
ax2.plot([xs, xe], [180, 180], 'k--')
ax2.set_xlabel("Distance Between Two Agents (World Units)")
ax2.set_ylabel("Angle Error of Acceleration (deg)")

ax1.set_title('Magnitude of Force (Hexagonal)')
ax2.set_title('Angle Error (Hexagonal)')     

plt.show()
